In [ ]:
import os
import json
import csv

import re

import  ftfy

from findJsonKeys import FindJsonKey 
from populateFlatByJson import populate ,cleanText


# transform comment file

In [13]:
facebook_comment_dir=os.environ["FACE_DATA"]+"your_activity_across_facebook/comments_and_reactions/"



findKeys=FindJsonKey()
def transform(msg:dict,d:dict)-> dict:
    result={}
    result["title"]=cleanText(msg["title"])


    populate(d)

    result["timestamp"] = msg["timestamp"]
    return result

all_data=[]    
for root,folders,files in os.walk(facebook_comment_dir):
    folder=root.replace(facebook_comment_dir,"")
    for file in [file for file in files if "comments" in file]:
        with open(os.path.join(root,file), 'r') as user_file:
            parsed_json = json.load(user_file)

            parsed_json = parsed_json["comments_v2"]
            attachments = [transform(msg,d) for msg in parsed_json if "attachments" in msg for a in msg["attachments"] for d in a['data']]
            comments =  [transform(msg,d["comment"]) for msg in parsed_json if "data" in msg for d in msg["data"]]
            
            all_data += (attachments)
            all_data += (comments)
                

print(findKeys.keys())      

with open(os.path.join(os.path.dirname(facebook_comment_dir),"comment.json"), 'w', newline='', encoding='utf-8') as jsonFile:                
    
    parsed_json = [item for item in all_data]
    json.dump(parsed_json,jsonFile,indent=2)             

# Transform Like File
Read all like files and stract the reaction person date. But I liket to extract more data soo with Regex I spit the action and the person.

In [ ]:


all_data=[]    
findKeys=FindJsonKey()
def transform(msg,data):
    
    result =  {
        "title":ftfy.fix_text(msg["title"]),
        'reaction':data["reaction"]["reaction"],
        'timestamp':msg['timestamp']     
    }

    itens=re.findall("^Mariano Aloi ([^']+)'*s* (.*).$",cleanText(msg["title"]))
    if len(itens) > 0 :
        itens=itens[0]
        result["typeItem"]=itens[1]  
        
        result["person"]=itens[0].replace( \
            "liked " if re.match("^liked .*",itens[0]) \
                else \
            "reacted to " if re.match("^reacted to .*",itens[0]) \
                else "","");
                             
    return result;

for root,folders,files in os.walk(facebook_comment_dir):
    folder=root.replace(facebook_comment_dir,"")
    for file in [file for file in files if "likes_and_reactions" in file]:
        with open(os.path.join(root,file), 'r') as user_file:
            parsed_json = json.load(user_file)
            # findKeys.makeDecision(parsed_json)
            parsed_json = parsed_json = [transform(msg,data) for msg in parsed_json for data in msg["data"]]
            all_data += (parsed_json)
print(findKeys.keys())


with open(os.path.join(os.path.dirname(facebook_comment_dir),"comment.json"), 'w', newline='', encoding='utf-8') as jsonFile:                
    
    parsed_json = [item for item in all_data]
    json.dump(parsed_json,jsonFile,indent=2)  